In [1]:
import group_data as gd
import flylib
import numpy as np
%pylab inline
%config InlineBackend.figure_format = 'svg'
import pylab as plb
plb.rcParams['pdf.fonttype'] = 42

Populating the interactive namespace from numpy and matplotlib


In [2]:
print 'here'

here


# EpochDB
stack all epochs and signals into single table, use vectors to describe each signal type and trial type. (epoch db)

In [3]:
######
#blurred background subtracted data
######

trial_numbers = list()
trial_names = list()
fly_numbers = list()
signal_units = list()
signal_data = list()
signal_names = list()
swarm_names = list()
trial_times = list()
exp_names = list()
genotypes = list()
flt_samples = list()

for swarm_name,swarm in gd.swarms.items():
    for fly in swarm.flies:
        try:
            sigs = fly.get_pkl_data('nnls_fits_bk_sub_F_sig_mtrx.cpkl')
            for i,trial in enumerate(sigs['StimCond']):
                for key in ['StimCond','wb_frequency',
                            'Ph0','Ph1','Ph2','Xpos','Ypos',
                            'b1','b2','b3',
                            'i1','i2',
                            'iii1','iii3','iii24',
                            'hg1','hg2','hg3','hg4',
                            'tpd','tpv','ttm','pr','nm']:
                    try:
                        sd = np.array(sigs[key][i])
                        tna = sigs['trial_cond'][i]
                        tnum = i
                        fnum = fly.fly_num
                        snam = key
                        swna = swarm_name
                        trti = sigs['trial_time'][i]
                        expna = str(fly.list_expmnts()[0])
                        geno = fly.get_genotype()
                        fsmp = np.nansum(sigs['wb_frequency'][i] > 50)
                        
                        trial_names.append(tna)
                        trial_numbers.append(tnum)
                        fly_numbers.append(fnum)
                        signal_names.append(snam)
                        swarm_names.append(swna)
                        trial_times.append(trti)
                        exp_names.append(expna)
                        signal_data.append(sd)
                        genotypes.append(geno)
                        flt_samples.append(fsmp)
                        
                    except KeyError as err:
                        pass
                        #print err
                        #print 'key error'
        except IOError:
            print fly.fly_num
            #print swarm_name
            #print 'missing file'
                #print 'missing: %s'%key
        except EOFError:
            print 'eof error' + str(fly.fly_num)
            
trial_numbers = np.array(trial_numbers)
trial_names = np.array(trial_names)
fly_numbers = np.array(fly_numbers)
signal_units = np.array(signal_units)
signal_data = np.array(signal_data)
signal_names = np.array(signal_names)
swarm_names = np.array(swarm_names)
trial_times = np.array(trial_times)
exp_names = np.array(exp_names)
genotypes = np.array(genotypes)
flt_samples = np.array(flt_samples)
import os
os.remove('epoch_db_nnls_fits_bk_sub_F.hdf5')
import h5py
epoch_db = h5py.File('epoch_db_nnls_fits_bk_sub_F.hdf5','a')
epoch_db.create_dataset('trial_numbers',data = trial_numbers,  compression="gzip",compression_opts=5)
epoch_db.create_dataset('trial_names',  data = trial_names,    compression="gzip",compression_opts=5)
epoch_db.create_dataset('fly_numbers',  data = fly_numbers,    compression="gzip",compression_opts=5)
epoch_db.create_dataset('signal_data',  data = signal_data,    compression="gzip",compression_opts=5)
epoch_db.create_dataset('signal_names', data = signal_names,    compression="gzip",compression_opts=5)
epoch_db.create_dataset('swarm_names',  data = swarm_names,    compression="gzip",compression_opts=5)
epoch_db.create_dataset('trial_times',  data = trial_times,    compression="gzip",compression_opts=5)
epoch_db.create_dataset('exp_names',    data = exp_names,    compression="gzip",compression_opts=5)
epoch_db.create_dataset('genotypes',    data = genotypes,    compression="gzip",compression_opts=5)
epoch_db.create_dataset('flt_samples',  data = flt_samples,  compression="gzip",compression_opts=5)
epoch_db.flush()
epoch_db.close()

648
370
631


/home/psilentp/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:39: RuntimeWarning: invalid value encountered in greater


# TrialDB
Create a series of tables for each signal type. Use vectors to indicate fly num, trial time, trial type ect.. (trial db)

In [ ]:
data_dict = dict()
#collumns of data
for key in ['StimCond','wb_frequency',
            'Ph0','Ph1','Ph2','Xpos','Ypos',
            'b1','b2','b3',
            'i1','i2',
            'iii1','iii3','iii24',
            'hg1','hg2','hg3','hg4',
            'tpd','tpv','ttm','nm','pr']:
    data_dict[key] = list()

data_dict['trial_numbers'] = list()
data_dict['trial_names'] = list()
data_dict['fly_numbers'] = list()
data_dict['signal_units'] = list()
#signal_names = list()
data_dict['swarm_names'] = list()
data_dict['trial_times'] = list()
data_dict['exp_names'] = list()
data_dict['genotypes'] = list()
#data_dict['flt_samples = list()

In [ ]:
#epoch_db_nnls_fits_bk_sub_F
lidx = 0
import ipywidgets as widgets
from IPython.display import display

nflies = 0
for swarm_name,swarm in gd.swarms.items():
    lidx +=1 
    for fly in swarm.flies:
        nflies +=1
        
pbar = widgets.FloatProgress(
    value=0,
    min=0.0,
    max=nflies,
    step=1,
    description='fitting:')

display(pbar)

xi = np.linspace(0,23,1150)
for swarm_name,swarm in gd.swarms.items():
    lidx +=1
    pbar.value = lidx
    for fly in swarm.flies:
        try:
            sigs = fly.get_pkl_data('nnls_fits_bk_sub_F_sig_mtrx.cpkl')
            #sigs = fly.get_pkl_data('nnls_fits_no_bk_dF_F_sig_mtrx.cpkl')
            for i,trial in enumerate(sigs['StimCond']):
                for key in ['StimCond','wb_frequency',
                            'Ph0','Ph1','Ph2','Xpos','Ypos',
                            'b1','b2','b3',
                            'i1','i2',
                            'iii1','iii3','iii24',
                            'hg1','hg2','hg3','hg4',
                            'tpd','tpv','ttm','pr','nm']:
                    try:
                        signal = np.array(sigs[key][i])
                    except KeyError:
                        signal = np.empty_like(xi)
                        signal[:] = np.NAN
                    data_dict[key].append(signal)
                try:
                    tna = sigs['trial_cond'][i]
                except KeyError:
                    tna = 'nan'
                try:
                    trti = sigs['trial_time'][i]
                except KeyError:
                    trti = np.NAN
                tnum = i
                fnum = fly.fly_num
                swna = swarm_name

                expna = str(fly.list_expmnts()[0])
                try:
                    geno = fly.get_genotype()
                except Exception as er:
                    #print er
                    geno = 'nan'
                data_dict['trial_numbers'].append(tnum)
                data_dict['trial_names'].append(tna)
                data_dict['fly_numbers'].append(fnum)
                data_dict['swarm_names'].append(swarm_name)
                data_dict['trial_times'].append(tna)
                data_dict['exp_names'].append(expna)
                data_dict['genotypes'].append(geno)
        except IOError:
            print fly.fly_num
            print swarm_name
            print 'missing file'
                #print 'missing: %s'%key
        except EOFError:
            print 'eof error' + str(fly.fly_num)

In [ ]:
import os
#os.listdir('./')
os.remove('trial_db.hdf5')
import h5py
trial_db = h5py.File('trial_db.hdf5','a')
for key in data_dict:
    trial_db.create_dataset(key,data = np.array(data_dict[key]),  compression="gzip",compression_opts=5)
trial_db.flush()

In [9]:
print 'here'

here
